In [60]:
import numpy as np
import pandas as pd 
from io import StringIO
import os
import warnings
warnings.filterwarnings('ignore')
from json import dumps, loads
import itertools 
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [61]:
def transform_to_json(result_df):
    result = result_df.to_json(orient="records", date_format='iso')
    return result

In [62]:
def predict_result(df):
    model = SARIMAX(df, order=(1, 1, 1), seasonal_order=(0, 1, 1, 12))
    results = model.fit()

    predictions = results.get_forecast(steps=2)
    
    mean_prediction = predictions.predicted_mean
    print(f'mean_prediction: {mean_prediction}')
    ci_95 = predictions.conf_int(alpha=0.05)
    result_df = pd.DataFrame({   
        'Forecast': mean_prediction,
        'Lower CI': ci_95.iloc[:, 0],
        'Upper CI': ci_95.iloc[:, 1]
    })
    return result_df

In [63]:
def fit_data(df):    
   
    df['dt'] = pd.to_datetime(df['dt'])
    df = df.set_index('dt')
#    column_name = df.columns[0]    
    return predict_result(df)   

In [64]:
#передается в след функцию dataframe и отдается наружу результат предсказания (сами значения forecast и доверительные интервалы)
def prediction(dt):   
    df = pd.DataFrame.from_dict(dt)
    result = fit_data(df)
    return result

In [ ]:
from flask import Flask, request, jsonify
import pandas as pd

app = Flask(__name__)

@app.route('/')
def home():
    return 'Welcome to our service!'
    
@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json() 
    #print(f'prediction(data):{prediction(data)}')    
    df = prediction(data)
    return transform_to_json(df)    
if __name__ == '__main__':
    app.run(port=8088, debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8088
Press CTRL+C to quit
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            4     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.46467D+00    |proj g|=  1.19592D-01

At iterate    5    f=  2.34861D+00    |proj g|=  9.17092D-03


127.0.0.1 - - [03/Sep/2024 17:41:45] "POST /predict HTTP/1.1" 200 -



At iterate   10    f=  2.34664D+00    |proj g|=  6.06305D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    4     12     15      1     0     0   7.122D-06   2.347D+00
  F =   2.3466432718985408     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
mean_prediction: 2024-01-01    10.649763
2024-02-01     8.156934
Freq: MS, Name: predicted_mean, dtype: float64


In [ ]:
prediction([
    {
        "dt": "2005-01-01T00:00:00.000", 
        "total_alcohol": 6.1684   
    },    
    {
        "dt": "2005-02-01T00:00:00.000",
        "total_alcohol": 5.1492
    }
])